In [1]:
library(gdata)
library(RColorBrewer)

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size



In [2]:
meta_data <- read.xls('metadata_updated_rates.xlsx', stringsAsFactors = F)
meta_data <- meta_data[-grep('N. meningitis', meta_data$Name), ]
meta_data <- meta_data[, -ncol(meta_data)]
meta_data$short_name <- gsub('_+$', '', gsub('ucld|_rc|_rs|_sc|_ss|_bs|constant|strict', '', meta_data$file_name))
head(meta_data[, c('beast_genomic_rate', 'Name')])
head(meta_data)


temp_struct <- meta_data$temp_stucture_degree >0.5
temp_struct[meta_data$temp_stucture_degree>=0.9] <- 'blue'
temp_struct[meta_data$temp_stucture_degree >= 0.5 & meta_data$temp_stucture_degree < 0.9] <- 'orange'
temp_struct[meta_data$temp_stucture_degree < 0.5] <- 'red'

,beast_genomic_rate,Name
1,1.562162e-08,M. leprae
2,1.572643e-08,Y. pestis
3,2.260972e-08,Y. pestis (second pandemic)
4,5.675213e-08,M. tuberculosis Lineage 4
6,7.604037e-08,S. Paratyphi A (two clades)
7,8.016475e-08,S. Paratyphi A (clade 1)


In [3]:
lm_randomisation <- function(rate, denominator, nreps = 1000, log_log = T){
    # Spurious correlations arise when y/x ~ x. 
        # To solve do
        # y/x * x = y
        # randomise y -> yr 1000 times
        # run regression using yr/x ~x: this is the null distribution
        true_lm <- lm(rate ~ denominator)
        if(log_log){
            get_rate <- function(a, b) return(a+b)
        }else{
            get_rate <- function(a, b) return(a*b)
        }
        y <- get_rate(rate, denominator)
        x <- denominator
        null_dist <- vector()
        for(i in 1:nreps){
            yr <- sample(x = y, size = length(y))
            yr.x <- get_rate(yr, x)
            null_dist[i] <- lm(yr.x ~ x)$coefficients[2]
        }
                
        return(list(0.5 - abs(sum(true_lm$coefficients[2] > null_dist)/nreps - 0.5), null_dist, true_lm$coefficients))
    }

In [4]:
pdf('genome_size_GC_content_time.pdf', useDingbats = F, width = 14, height = 5)
par(mfrow = c(1, 3))
par(mar = c(4, 4, 2, 2))

pass_cr1 <- (meta_data$temp_stucture_degree > 0.5)
drt <- c(21, 20)[pass_cr1 + 1]
#gram_code <- c(rgb(1,0,0.7, 1), rgb(0, 0, 1, 0.7))[meta_data$gram_coded]
#cols <- meta_data$species
plot(log10(meta_data$gsize_check), log10(meta_data$beast_genomic_rate), pch = 20, col = temp_struct, xlim = c(6.15, 6.8),
     ylab = 'Genomic rate (subs/site/year)', xlab = 'Genome size (sites)', cex = 2.5) #col = gram_code, 
genome_size_reg <- lm(log10(meta_data$beast_genomic_rate)[pass_cr1] ~  log10(meta_data$gsize_check)[pass_cr1])
abline(genome_size_reg)
text(log10(meta_data$gsize_check), log10(meta_data$beast_genomic_rate), 
     labels = meta_data$code, cex = 0.9, pos = 2 )#, vfont = c('sans serif', 'italic'))
print(summary(genome_size_reg))

plot(meta_data$CG., log10(meta_data$beast_genomic_rate), pch = 20, col = temp_struct, 
      xlab = 'GC content (%)', ylab = '', cex = 2.5, yaxt='n') 
cg_reg <- lm(log10(meta_data$beast_genomic_rate)[pass_cr1] ~ meta_data$CG.[pass_cr1])
abline(cg_reg)
text(meta_data$CG., log10(meta_data$beast_genomic_rate),
     labels = meta_data$code, cex = 0.9, pos = 2)#, vfont = c('sans serif', 'italic'))
print(summary(cg_reg))


plot(log10(meta_data$sampling_time), log10(meta_data$beast_genomic_rate), pch = 20, col = temp_struct, 
      xlab = 'Sampling time (years)', ylab = '', cex = 2.5, xlim = c(0, 3.3), yaxt='n') #col = gram_code,
text(log10(meta_data$sampling_time), log10(meta_data$beast_genomic_rate), 
     labels = meta_data$code, cex = 0.9, pos = 2)#, vfont = c('sans serif', 'italic'))


sampling_time_reg <- lm(log10(meta_data$beast_genomic_rate)[pass_cr1] ~ log10(meta_data$sampling_time)[pass_cr1]) 
abline(sampling_time_reg)
print(summary(sampling_time_reg))
random_time_reg <- lm_randomisation(rate = log10(meta_data$beast_genomic_rate)[pass_cr1], denominator = log10(meta_data$sampling_time)[pass_cr1])

dev.off()


Call:
lm(formula = log10(meta_data$beast_genomic_rate)[pass_cr1] ~ 
    log10(meta_data$gsize_check)[pass_cr1])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.23574 -0.54513  0.03018  0.47987  0.96910 

Coefficients:
                                       Estimate Std. Error t value Pr(>|t|)  
(Intercept)                              3.0310     4.2188   0.718   0.4791  
log10(meta_data$gsize_check)[pass_cr1]  -1.4396     0.6469  -2.225   0.0353 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.5731 on 25 degrees of freedom
Multiple R-squared:  0.1653,	Adjusted R-squared:  0.132 
F-statistic: 4.953 on 1 and 25 DF,  p-value: 0.0353


Call:
lm(formula = log10(meta_data$beast_genomic_rate)[pass_cr1] ~ 
    meta_data$CG.[pass_cr1])

Residuals:
    Min      1Q  Median      3Q     Max 
-1.4204 -0.2225  0.1422  0.3224  0.5830 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)             -4

pdf 
  2

In [8]:
boot_data <- read.table('curve_fit_bootstrap.csv', sep = ',', head =T)

In [23]:
pdf('rate_time.pdf', useDingbats = F, width = 7, height = 7)
#par(mfrow = c(1, 2))
par(mar = c(4, 4, 2, 2))

pass_cr1 <- (meta_data$temp_stucture_degree > 0.5)
drt <- c(21, 20)[pass_cr1 + 1]
#gram_code <- c(rgb(1,0,0.7, 1), rgb(0, 0, 1, 0.7))[meta_data$gram_coded]
#cols <- meta_data$species
#plot(log10(meta_data$gsize_check), log10(meta_data$beast_genomic_rate), pch = 20, col = temp_struct, xlim = c(6.15, 6.8),
#     ylab = 'Genomic rate (subs/site/year)', xlab = 'Genome size (sites)', cex = 2.5) #col = gram_code, 
#genome_size_reg <- lm(log10(meta_data$beast_genomic_rate)[pass_cr1] ~  log10(meta_data$gsize_check)[pass_cr1])
#abline(genome_size_reg)
#text(log10(meta_data$gsize_check), log10(meta_data$beast_genomic_rate), 
#     labels = meta_data$code, cex = 0.9, pos = 2 )#, vfont = c('sans serif', 'italic'))
#print(summary(genome_size_reg))

#plot(meta_data$CG., log10(meta_data$beast_genomic_rate), pch = 20, col = temp_struct, 
#      xlab = 'GC content (%)', ylab = '', cex = 2.5, yaxt='n') 
#cg_reg <- lm(log10(meta_data$beast_genomic_rate)[pass_cr1] ~ meta_data$CG.[pass_cr1])
#abline(cg_reg)
#text(meta_data$CG., log10(meta_data$beast_genomic_rate),
#     labels = meta_data$code, cex = 0.9, pos = 2)#, vfont = c('sans serif', 'italic'))
#print(summary(cg_reg))

plot(log10(meta_data$sampling_time), log10(meta_data$beast_genomic_rate), pch = 20, col = temp_struct, 
      xlab = 'Sampling time (years)', cex = 2.5, xlim = c(0, 3.3), 
     ylab = 'Genomic rate (subs/site/year)', type = 'n') #col = gram_code,
text(log10(meta_data$sampling_time), log10(meta_data$beast_genomic_rate), 
     labels = meta_data$code, cex = 0.9, pos = 2)#, vfont = c('sans serif', 'italic'))

####
####
#### These lines represent the y=a/x^b fitted in notebook fit_rate_decay
for(i in 1:nrow(boot_data)){
    line_temp <- boot_data$alpha[i]/(sort(log10(meta_data$sampling_time))^boot_data$beta[i])
    lines(sort(log10(meta_data$sampling_time)), line_temp, col = rgb(0, 0, 0.3, 0.01), lwd = 4)
}
fun_mle <- function(x) -5.9/(x^-0.17)
lines(sort(log10(meta_data$sampling_time)), fun_mle(sort(log10(meta_data$sampling_time))),
      lwd = 3)
####
####
####
points(log10(meta_data$sampling_time), log10(meta_data$beast_genomic_rate), 
       pch = 20, col = temp_struct, cex = 2.5)
    
sampling_time_reg <- lm(log10(meta_data$beast_genomic_rate)[pass_cr1] ~ log10(meta_data$sampling_time)[pass_cr1]) 

b <- sampling_time_reg$coefficients[1]
m <- sampling_time_reg$coefficients[2]
get_y <- function(x) as.numeric(m*x+b)

lines(range(log10(meta_data$sampling_time)),             
            sapply(range(log10(meta_data$sampling_time)), get_y ), lwd = 2, lty = 2)
    
text(x = 2.5, y = -5.5, lab= 'Slope = -0.707')
text(x = 2.5, y = -5.8, lab= 'P = 0.0003')

    

#abline(sampling_time_reg)

    print(summary(sampling_time_reg))
random_time_reg <- lm_randomisation(rate = log10(meta_data$beast_genomic_rate)[pass_cr1], denominator = log10(meta_data$sampling_time)[pass_cr1])

dev.off()


Call:
lm(formula = log10(meta_data$beast_genomic_rate)[pass_cr1] ~ 
    log10(meta_data$sampling_time)[pass_cr1])

Residuals:
     Min       1Q   Median       3Q      Max 
-1.18481 -0.34918  0.01452  0.34031  0.69120 

Coefficients:
                                         Estimate Std. Error t value Pr(>|t|)
(Intercept)                               -5.2694     0.2778 -18.969 2.35e-16
log10(meta_data$sampling_time)[pass_cr1]  -0.7072     0.1706  -4.145 0.000341
                                            
(Intercept)                              ***
log10(meta_data$sampling_time)[pass_cr1] ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4829 on 25 degrees of freedom
Multiple R-squared:  0.4073,	Adjusted R-squared:  0.3836 
F-statistic: 17.18 on 1 and 25 DF,  p-value: 0.000341



pdf 
  2

In [6]:

sapply(range(log10(meta_data$sampling_time)), get_y )

[1] -5.504509 -7.602923

In [37]:
range(log10(meta_data$sampling_time))

[1] 0.3324385 3.2995073

In [2]:
-5.2694+(0.277*1.96)
-5.2694-(0.277*1.96)

[1] -4.72648

[1] -5.81232